In [1]:
import urllib
from bs4 import BeautifulSoup
%matplotlib inline
import matplotlib
import shapefile
import pandas as pd
import re
from shapely import geometry
import os
import seaborn
from collections import defaultdict
from shapely import ops
import pickle
import matplotlib.pyplot  as plt
from shapely import speedups
import numpy as np
import pandas as pd
#from yelp.client import Client
#from yelp.oauth1_authenticator import Oauth1Authenticator
#from haversine import haversine
import numpy as np
import math
import shapely
import shapefile
import os
import scipy as sp
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import datetime
import seaborn as sns
#from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import glob
import shapefile
#import mpl_toolkits.basemap.pyproj as pyproj
from sklearn import cluster
from IPython import display
from collections import defaultdict
import pickle
import json
import utm
from collections import defaultdict
from numpy import linalg
from itertools import combinations


In [2]:
speedups.enable()

# First we will extract the necessary data from the shapefile

In [3]:
sf = shapefile.Reader("new-york_new-york.imposm-shapefiles/new-york_new-york_osm_admin.shp")
sf_ref = sf.shapeRecords()
names = {"Kings County":"Brooklyn","Queens County":"Queens","Richmond County":"Statten Island",
         "New York County":"Manhattan","Bronx County":"Bronx"}
ny_records = {}
polygons = []
for rec in sf_ref:
    name = rec.record[2]
    if rec.record[-1] == 6 and name in names:
        poly = geometry.Polygon(rec.shape.points)
        if not poly.is_valid:
            poly = poly.buffer(0)[1]
        ny_records[names[name]] = poly

Self-intersection at or near point -74.039740173399863 40.699024314884056
Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/logging/__init__.py", line 874, in emit
    stream.write(fs % msg)
IOError: [Errno 5] Input/output error
Logged from file geos.py, line 230


In [4]:
minLat = np.inf
maxLat = -np.inf
minLng = np.inf
maxLng = -np.inf
for bur in ny_records:
    lng,lat = ny_records[bur].boundary.xy
    lngMin,lngMax = min(lng),max(lng)
    latMin,latMax = min(lat), max(lat)
    if lngMin < minLng:
        minLng = lngMin
    if lngMax > maxLng:
        maxLng = lngMax
    if latMin < minLat:
        minLat = latMin
    if latMax > maxLat:
        maxLat = latMax

In [5]:
def get_grid(spacing):
    latNums = int((maxLat-minLat)/spacing)
    lngNums =int((maxLng-minLng)/spacing)
    latAr = np.linspace(minLat-10**(-10),maxLat+10**(-10), latNums)
    lngAr= np.linspace(minLng-10**(-10), maxLng+10**(-10),lngNums)
    boxes = []
    for i in range(len(lngAr)-1):
        for j in range(len(latAr)-1):
            boxes.append(geometry.box(lngAr[i],latAr[j],lngAr[i+1],latAr[j+1]))
    return (np.array(boxes).reshape(len(lngAr)-1, len(latAr)-1),latAr,lngAr)

In [6]:
def get_relevant(boxes):
    relevant = []
    full_NYC = ops.unary_union([ny_records[val] for val in ny_records])
    for box in boxes.flatten():
        if box.intersects(full_NYC):
            keep = True
            relevant.append(box.intersection(full_NYC))
        else:
            relevant.append(None)
    return np.array(relevant).reshape(boxes.shape)

In [24]:
SPACING = 0.005/4

In [26]:
BOXES,latAr,lngAr = get_grid(SPACING)

In [27]:
RELEVANT = get_relevant(BOXES)

In [7]:
def get_index(lat,lng,latAr,lngAr):
    jLat = np.argmax(lat <= latAr)-1
    iLng = np.argmax(lng<=lngAr)-1
    return (iLng,jLat)
def get_burough(coord,records):
    #lat_str = "_latitude"
    #lng_str = "_longitude"
    #NYC = get_NYC_records(sf)
    #shapes = sf.shapes()
    point = geometry.Point(coord[::-1])

    for bur, poly in records.items():
        #if poly[0].contains(point):
        if poly.contains(point):
            return bur
    return None
def get_all_burough(df, drop_or_pick = "dropoff"):
    coords = df[[drop_or_pick + "_lat",drop_or_pick +"_long"]].values
    return map(lambda x: get_burough(x,ny_records),coords)

In [8]:
def get_all_boxes(df,latAr,lngAr):
    coord_pick = df[["pickup_lat", "pickup_long"]].values
    coord_drop = df[["dropoff_lat", "dropoff_long"]].values
    pick_vals = pd.DataFrame({"pick_grid":map(lambda x: get_index(*x,latAr = latAr,lngAr = lngAr), coord_pick)})
    drop_vals = pd.DataFrame({"drop_grid":map(lambda x: get_index(*x,latAr = latAr,lngAr = lngAr), coord_drop)})
    return pd.concat([pick_vals,drop_vals], axis = 1)

In [9]:
def get_rest_boxes(df,latAr,lngAr):
    coords = df[["Lat","Long"]].values
    return pd.DataFrame({"grid_point":map(lambda x: get_index(*x,latAr = latAr,lngAr = lngAr), coords)})

# Global Varibles

In [12]:
nyc_centroid = {"lat":40.6639206199602,"lng": -73.9383529238219}
bounds_of_nyc = {"lat":(minLat,maxLat), "lng":(minLng, maxLng)}

# Useful functions

In [13]:
def convert_datetime(df):
    cols = ["pickup_datetime","dropoff_datetime"]
    for col in cols:
        df.loc[:,col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
def half_hour_grading(date_time):
    time = date_time.time()
    h,m = time.hour,time.minute
    if m < 30:
        m = 0
    else:
        m = 0.5
    return h + m

def add_details(df):
    #df.loc[:,"pickup_year"] = df["pickup_datetime"].apply(lambda x: x.year)
    df.loc[:,"pickup_month"] = df["pickup_datetime"].apply(lambda x: x.month)
    df.loc[:,"pickup_day"] = df["pickup_datetime"].apply(lambda x: x.weekday())
    df.loc[:,"duration"] = df["dropoff_datetime"]-df["pickup_datetime"]
    df.loc[:,"time_category_pick"] = df["pickup_datetime"].apply(lambda x: x.hour)
    df.loc[:,"time_category_drop"] = df["dropoff_datetime"].apply(lambda x: x.hour)
    df.loc[:,"half_hour_pick"] = df["pickup_datetime"].apply(lambda x: half_hour_grading(x))
    df.loc[:,"half_hour_drop"] = df["dropoff_datetime"].apply(lambda x: half_hour_grading(x))
    


In [14]:
def clean_data(df, bounds, remove_airport = True):
    min_lat, max_lat = bounds["lat"]
    min_long, max_long = bounds["lng"]
    dfFilter = df[(min_lat <= df.pickup_lat) &(df.pickup_lat <= max_lat) & (min_long <= df.pickup_long) \
                  &(df.pickup_long<= max_long)]
    dfFilter = dfFilter[(min_lat <= dfFilter.dropoff_lat) &(dfFilter.dropoff_lat <= max_lat) \
                       & (min_long <= dfFilter.dropoff_long)  &(dfFilter.dropoff_long<= max_long)]
    dfFilter = dfFilter[dfFilter.duration >= datetime.timedelta(0)]
    #if remove_airport:
    #    dfFilter = dfFilter[dfFilter.rate_code.isin([1,5,6])]
    return dfFilter.reset_index(drop = True)

In [32]:
def get_coords(df):
    pick_result = {}
    drop_result = {}
    for name,group in df:
        pick_result[name] = zip(group.pickup_lat.values, group.pickup_long.values)
        drop_result[name] = zip(group.dropoff_lat.values, group.dropoff_long.values)
    return (pick_result, drop_result)
def convert_to_xy_list(coord_list):
    xy_list = map(lambda x: utm.from_latlon(*x), coord_list)
    return xy_list
def convert_to_xy(coord_dict):
    return dict([(key,convert_to_xy_list(coord_dict[key]))for key in coord_dict])
def strip_xy(coord_list):
    return [val[:2] for val in coord_list]

In [ ]:
def make_dict(groups, coords):
    res = defaultdict(list)
    for group, coord in zip(groups, coords):
        if group != -1:
            res[group].append(utm.to_latlon(*coord))
    return dict(res)

In [ ]:
def clusterCOORD(coordsXY,throttle = 0.01,metric ="euclidean"):
    res = {}
    #display.display(eps)
    #display.display(min_samp)
    for date in coordsXY:
        if date[0] == "Manhattan":
            eps = 50
            min_samples = max(2,len(coordsXY[date])*throttle)
        #display.display(date)
        else:
            eps = 500
            min_samples = max(2,len(coordsXY[date])*throttle)
        dbScan = cluster.DBSCAN(eps = eps, min_samples= min_samples, metric = metric)
        result = dbScan.fit_predict(strip_xy(coordsXY[date]))
        
        res[date] = make_dict(result, coordsXY[date])
        
    return res

# Process the Data

In [15]:
url = "http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml"
r = urllib.urlopen(url)
soup = BeautifulSoup(r, 'lxml')

In [18]:
chosen_names_2015 = ["vendorID", "pickup_datetime", "dropoff_datetime","passenger_count", "trip_distance",\
                          "pickup_long","pickup_lat","rate_code","store_and_forward","dropoff_long","dropoff_lat",\
                          "payment_type","fare_amount","surcharge","mta_tax","tip_amount","tolls_amount",\
                          "improvement_surcharge","total_amount"]

In [19]:
def get_urls(soup):
    for a in soup.findAll('a',href = True):
        ref = a['href']
        if "tlc-trip-data" in ref :
            #test = urllib.URLopener()
            #test.retrieve(ref, "/Volumes/My Passport/Taxi_Data/Yellow_Taxi/yellow_tripdata_2014-01.csv")
            name = ref.split('/')[-1]
            color = name.split('_')[0]
            year = name.split("_")[-1].split("-")[0]
            color = color[0].upper() + color[1:]+ "_Taxi"
            #print name, color
            file_path = color +"/" + name
            if int(year) >=2015 and color == 'Yellow_Taxi':
                yield (ref,name.split("-")[-1][:2])
def turn_into_panda(soup):
    for url in get_urls(soup):
        yield (pd.read_csv(url)

In [20]:
def reform_data_frame(soup):
    for df in turn_into_panda(soup):
        df.columns = chosen_names_2015
        convert_datetime(df)
        add_details(df)
        df = clean_data(df, bounds_of_nyc)
        yield df
        #add_details
        

In [29]:
def get_burough_info(df,soup,spacing):
    #for df in reform_data_frame(soup):
    gridVals = get_all_boxes(df,latAr,lngAr).reset_index(drop = True)
    burVals = pd.Series(get_all_burough(df))
    burVal = pd.DataFrame({"bur":burVals}).reset_index(drop = True)
    df = df.reset_index(drop = True)
    gridVals = gridVals.reset_index(drop = True)

    return pd.concat([df, burVals,gridVals], axis = 1)
        
        

In [45]:
df.columns

Index([             u'vendorID',       u'pickup_datetime',
            u'dropoff_datetime',       u'passenger_count',
               u'trip_distance',           u'pickup_long',
                  u'pickup_lat',             u'rate_code',
           u'store_and_forward',          u'dropoff_long',
                 u'dropoff_lat',          u'payment_type',
                 u'fare_amount',             u'surcharge',
                     u'mta_tax',            u'tip_amount',
                u'tolls_amount', u'improvement_surcharge',
                u'total_amount',           u'pickup_year',
                u'pickup_month',            u'pickup_day',
                    u'duration',    u'time_category_pick',
          u'time_category_drop',        u'half_hour_pick',
              u'half_hour_drop',                        0,
                   u'pick_grid',             u'drop_grid'],
      dtype='object')

In [30]:
df = get_burough_info(df, soup,SPACING)

In [ ]:
df.groupby("ti")

In [48]:
get_heat_map_params(df, "tip_amount",0.1,"Jan")

/opt/conda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: index 450 is out of bounds for axis 0 with size 446

In [47]:
def get_heat_map_params(df,item, blur,name):
    MAX_VAL = np.log(np.log(df.groupby(item).agg('size').max()+1)+1)+0.000001
    counts = df.groupby(item).agg('size')
    #maxVal = counts.max() + 0.000001
    vals = np.zeros(shape = boxes.shape)
    for i,val in enumerate(counts.index):
        vals[val] = np.log(np.log(counts.values[i]+1)+1)/MAX_VAL
    with open(name + "_cmap_"+str(item)+".dill","w") as f:
        dill.dump(gaussian_filter(vals,sigma=blur),f)

In [ ]:
def cluster_step1_get_coords(df):
    Grouped = df.groupby(by = ["bur","pickup_year","pickup_month","pickup_day","time_category_drop"])
    pick_coords, drop_coords = get_coords(df)
    coords_pick_xy = convert_to_xy(pick_coords)
    coords_drop_xy = convert_to_xy(drop_coords)
    return (coords_pick_xy,coords_drop_xy)

In [ ]:
df

In [22]:
dfIter = reform_data_frame(soup)

In [23]:
df = dfIter.next()

In [39]:
%reset pandas

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.
